# 1.9 Proyecto de Cierre (Nivel Junior)\n\n- Objetivo: integrar SELECT, JOIN, filtros, agregaciones y subconsultas.\n- Prerrequisitos: dataset preparado, Level01 1.1–1.8 completado.\n- Entregables: consultas para tablero ejecutivo básico.\n- Reto: consolidar con CTEs y controles de calidad.\n- Errores comunes: mezclar granularidad, filtros inconsistentes, duplicidades en agregaciones.

## Enunciado

Eres analista junior. El equipo comercial necesita un mini dashboard con: 
1. Ventas totales y margen total.
2. Top 5 productos por unidades.
3. Clientes nuevos últimos 30 días.
4. Ticket promedio por región.
5. Productos sin ventas (catalogados pero nunca vendidos).
6. Ventas con anomalías (descuento >50% o cantidad <=0).
7. Participación de cada segmento (por unidades y por margen).

🟢 Paso 1: Métricas globales.

In [ ]:
SELECT SUM(cantidad) AS unidades_totales,
       SUM(cantidad * precio_unitario * (1 - descuento_pct/100.0)) AS ingresos_est,
       SUM(cantidad * (precio_unitario - costo_unitario)) AS margen_total
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id;

🟢 Paso 2: Top 5 productos.

In [ ]:
SELECT TOP 5 p.producto_id, p.nombre, SUM(fv.cantidad) AS unidades
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY p.producto_id, p.nombre
ORDER BY unidades DESC;

🟠 Paso 3: Clientes nuevos últimos 30 días.

In [ ]:
SELECT cliente_id, nombre, fecha_alta
FROM dbo.dim_clientes
WHERE fecha_alta >= DATEADD(DAY,-30,CAST(GETDATE() AS DATE));

🟠 Paso 4: Ticket promedio por región.

In [ ]:
SELECT r.nombre_region,
       AVG(fv.cantidad * p.precio_unitario * (1 - descuento_pct/100.0)) AS ticket_promedio
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
JOIN dbo.dim_regiones r ON c.region_id = r.region_id
GROUP BY r.nombre_region;

🟠 Paso 5: Productos sin ventas.

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
LEFT JOIN dbo.fact_ventas fv ON p.producto_id = fv.producto_id
WHERE fv.producto_id IS NULL;

🔴 Paso 6: Ventas con anomalías.

In [ ]:
SELECT venta_id, fecha, producto_id, cliente_id, cantidad, descuento_pct
FROM dbo.fact_ventas
WHERE descuento_pct > 50 OR cantidad <= 0;

🔴 Paso 7: Participación por segmento (unidades y margen).

In [ ]:
SELECT c.segmento,
       SUM(fv.cantidad) AS unidades,
       SUM(fv.cantidad * (p.precio_unitario - p.costo_unitario)) AS margen
FROM dbo.fact_ventas fv
JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY c.segmento;

⚙️ Desafío Senior (Opcional): Crear una sola consulta con CTEs que devuelva todas las tablas anteriores en salida separada usando múltiples CTE y SELECT finales etiquetados.

In [ ]:
-- Sugerencia de estructura de CTEs (no ejecutar si no se desea):
-- WITH ventas_globales AS ( ... ), top_productos AS ( ... ), clientes_nuevos AS ( ... ),
-- ticket_region AS ( ... ), sin_ventas AS ( ... ), anomalías AS ( ... ), segmento AS ( ... )
-- SELECT * FROM ventas_globales;
-- SELECT * FROM top_productos;
-- ...

Errores comunes: no aislar filtros; duplicar productos en agregaciones; inconsistencia de fechas.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](08_calidad_datos_basica.ipynb)
---
